In [2]:
import numpy as np
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, TimeDistributed
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model
%matplotlib inline
import json
import pandas as pd
from keras.models import Sequential

from utils import *

pd.options.display.max_columns = None
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
data_header='target'
for i in range(2808): data_header += ',' + str(i)

#### Get data

In [22]:
# dataset


fpaths = []
labels = []
boy_fpaths = []
boy_labels = []

word_spoken = []


dataset='new'

dataset = '84{}'.format(dataset)
input_folder = '..\data\{}'.format(dataset)

# pars the input directory that contains audio files
# get audio files and their lables

for f in os.listdir(input_folder):
    for w in os.listdir(input_folder+'\\'+ f):
        # check wheter files is wav or not

        if (w.find('wav')!=-1):
            
            fpaths.append(input_folder+'\\'+f+'\\'+w)
            labels.append(f)
        

#             idx_schunk=w.find('schunk')
#             idx_chunk=w.find('chunk')

#             if (idx_chunk!=idx_schunk==-1):
#                 if (int(w[idx_chunk-2:idx_chunk])<80):
#                     fpaths.append(input_folder+'\\'+f+'\\'+w)
#                     labels.append(f)
#                 else:
#                     boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
#                     boy_labels.append(f)

#             else:
#                 boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
#                 boy_labels.append(f)


        if f not in word_spoken:
            word_spoken.append(f)
print("Spoken words: "+ str(word_spoken))

Spoken words: ['acknowledge', 'affirm', 'airborne', 'alpha', 'altitude', 'approved', 'arrival', 'backtrack', 'base_leg', 'bravo', 'break', 'cancel', 'charlie', 'check', 'cleared', 'climb', 'confirm', 'contact', 'control', 'correct', 'correction', 'degree', 'delta', 'departure', 'descend', 'direct_to', 'disregard', 'echo', 'emergency', 'fan_stop', 'foxtrot', 'golf', 'go_ahead', 'ground', 'hold_over', 'hold_short', 'hotel', 'identified', 'india', 'information', 'juliet', 'kilo', 'knots', 'landed_at', 'lima', 'line_up', 'maintain', 'main_engine', 'mike', 'monitor', 'negative', 'november', 'oscar', 'out', 'over', 'papa', 'point', 'port_engine', 'quebec', 'read_back', 'report', 'request', 'roger', 'romeo', 'runway', 'say_again', 'sierra', 'stand', 'standby', 'start_up', 'tango', 'taxi', 'taxiway', 'time', 'turn_left', 'turn_right', 'unable', 'uniform', 'victor', 'whiskey', 'wilco', 'wind', 'x-ray', 'yankee']


In [23]:
print("Number of classes: " + str(len(word_spoken)))

Number of classes: 84


In [24]:
# size of dataset
print(len(labels))

38602


In [25]:
label_set=list(set(labels))
label_set.sort()

# # boys 
# boy_label_set=list(set(boy_labels))
# boy_label_set.sort()

In [26]:
label_set

['acknowledge',
 'affirm',
 'airborne',
 'alpha',
 'altitude',
 'approved',
 'arrival',
 'backtrack',
 'base_leg',
 'bravo',
 'break',
 'cancel',
 'charlie',
 'check',
 'cleared',
 'climb',
 'confirm',
 'contact',
 'control',
 'correct',
 'correction',
 'degree',
 'delta',
 'departure',
 'descend',
 'direct_to',
 'disregard',
 'echo',
 'emergency',
 'fan_stop',
 'foxtrot',
 'go_ahead',
 'golf',
 'ground',
 'hold_over',
 'hold_short',
 'hotel',
 'identified',
 'india',
 'information',
 'juliet',
 'kilo',
 'knots',
 'landed_at',
 'lima',
 'line_up',
 'main_engine',
 'maintain',
 'mike',
 'monitor',
 'negative',
 'november',
 'oscar',
 'out',
 'over',
 'papa',
 'point',
 'port_engine',
 'quebec',
 'read_back',
 'report',
 'request',
 'roger',
 'romeo',
 'runway',
 'say_again',
 'sierra',
 'stand',
 'standby',
 'start_up',
 'tango',
 'taxi',
 'taxiway',
 'time',
 'turn_left',
 'turn_right',
 'unable',
 'uniform',
 'victor',
 'whiskey',
 'wilco',
 'wind',
 'x-ray',
 'yankee']

In [27]:
len(label_set)

84

## Extracting frequeny domain features

At the second stage we convet a signal into the freqency domain. In monst modern speech recognitoon freqeency-domain features are used as key component. In case of multispeakers MFFC feature extraction works best. After convert a signal into a freq domain, it's requered to convert it into a useable form. **Mel Frequency Cepstral Coefficients (MFCC)** is a good way to do that. *MFCC* takes the power spectrum of a signal and then uses a combination of filter banks and disrete cosinetransform to extract pattern of phones or features.

After extracting **MFFC** features we exract data into single data matrix, and a label vector with the correct label for eac data file is ceated. 

In [28]:
from scipy.io import wavfile

def get_mfcc(fpaths):
    data = []

    mfcc_max_length = 0

    # first file desitination name and index
    file_name = ''
    word_spoken_index = 0
    
    count=0;

    for n,file in enumerate(fpaths):

        # show current desintation 
        if (file.find(file_name)<=0):
            file_name=word_spoken[word_spoken_index]
            print(word_spoken[word_spoken_index])
            word_spoken_index+=1

        # read file 
        sampling_freq, audio = wavfile.read(file)
        # Extract MFCC features
        mfcc_features = mfcc(audio, sampling_freq)
    
        mfcc_len=mfcc_features.shape[0]*mfcc_features.shape[1]
        # get length of largets feature array
        if mfcc_len>mfcc_max_length:    
            mfcc_max_length=mfcc_len


        # flat data into 2D array
        mfcc_features=np.resize(mfcc_features,(1,mfcc_len))

        data.insert(n,mfcc_features)
        
    return data, mfcc_max_length

In [29]:
data, max_len=get_mfcc(fpaths)
# boy_data=get_mfcc(boy_fpaths)

acknowledge
affirm
airborne
alpha
altitude
approved
arrival
backtrack
base_leg
bravo
break
cancel
charlie
check
cleared
climb
confirm
contact
control
correct
correction
degree
delta
departure
descend
direct_to
disregard
echo
emergency
fan_stop
foxtrot
golf
go_ahead
ground
hold_over
hold_short
hotel
identified
india
information
juliet
kilo
knots
landed_at
lima
line_up
maintain
main_engine
mike
monitor
negative
november
oscar
out
over
papa
point
port_engine
quebec
read_back
report
request
roger
romeo
runway
say_again
sierra
stand
standby
start_up
tango
taxi
taxiway
time
turn_left
turn_right
unable
uniform
victor
whiskey
wilco
wind
x-ray
yankee


In [30]:
print("Lenght Longest Audio Array: " + str(max_len))

Lenght Longest Audio Array: 4914


#### Get Labels

In [31]:
def get_labels(labels, label_set):
    label_idx_dic={} # dict for storing labels

    #Each sample file is one row in data, and has one entry in labels
    print('Number of files total:', len(labels))
    all_labels = np.zeros(len(labels),dtype=int)
    for n, l in enumerate(label_set):
        label_idx_dic[n]=l
        all_labels[np.array([i for i, _ in enumerate(labels) if _ == l])] = n
    print('Labels and label indices', all_labels)
    
    return label_idx_dic, all_labels

In [32]:
label_dic, all_label = get_labels(labels,label_set)
# boy_label_dic, boy_all_label = get_labels(boy_labels,boy_label_set)

Number of files total: 38602
Labels and label indices [ 0  0  0 ... 83 83 83]


In [33]:
# save label dictionary
def save_dict_to_file(dic,dataset):
    with open('dict{}.txt'.format(dataset),'w') as file:
        file.write(json.dumps(dic))

In [34]:
# save dict
save_dict_to_file(label_dic,dataset)

#### Pad Zeros to Small Arrays

In [41]:
def data_x_y(data,all_labels):
    x_data = np.zeros((len(data),2808+1), dtype=float)

    for i,_d in enumerate(data):

        if (_d.shape[1]>2808):
            x_data[i,1:]=_d[0,0:2808]
        else :
            x_data[i,1:_d.shape[1]+1]=_d

        x_data[i,0]=all_labels[i]
        
    return x_data

#### SAVE DATASET

In [42]:
file_name="84new"

In [43]:
# save all features data
np.savetxt('data/{}.csv'.format(file_name), data_x_y(data,all_label), delimiter=',',header=data_header)

# np.savetxt('data/{}/data41{}.csv'.format(save_path,'ADA_boy'), data_x_y(boy_data,boy_all_label), delimiter=',',header=data_header)
# np.savetxt('data/{}/data41{}.csv'.format(save_path,'ADA_girl'), data_x_y(data,all_label), delimiter=',',header=data_header)

#### GENERATE  MIX NEW DATASET

In [4]:
features_df = pd.read_csv('data/data41.csv')
features_df_1 = pd.read_csv('data/data41ADA.csv')
features_df_2 = pd.read_csv('data/new_test/data41new_test.csv')
features_df_3 = pd.read_csv('data/new_test/data41new_test2.csv')

In [5]:
x_data = np.vstack((features_df, features_df_1,features_df_2,features_df_3))

In [6]:
print("normal dataset shape: {}".format(features_df.shape))
print("ada dataset shape: {}".format(features_df_1.shape))
print("ada dataset shape: {}".format(features_df_2.shape))
print("ada dataset shape: {}".format(features_df_3.shape))
print("new mix dataset shape: {}".format(x_data.shape))

normal dataset shape: (21433, 2809)
ada dataset shape: (9187, 2809)
ada dataset shape: (3666, 2809)
ada dataset shape: (411, 2809)
new mix dataset shape: (34697, 2809)


In [7]:
save_path=""
np.savetxt('data/{}/data41{}.csv'.format(save_path,'mix2'), x_data, delimiter=',',header=data_header)

#### SPLIT TEST AND TRAIN DATA

In [44]:
save_path="84new"

In [45]:
x_data = pd.read_csv('data/84new.csv')
x_data.tail(5)

# target         0          1          2          3          4  \
38597      83.0 -2.446586 -29.414751  -2.020952  15.708614 -44.844125   
38598      83.0 -1.356830 -34.434559  10.433897  36.797872 -48.981364   
38599      83.0 -1.625806 -36.543782  10.397370  31.705596 -51.939008   
38600      83.0 -1.408552 -33.430009  -2.378085  19.410358 -44.692626   
38601      83.0 -1.752262 -34.452788   4.242038  29.660638 -60.179196   

               5          6          7          8         9         10  \
38597 -22.539966 -32.319106   3.858966 -32.770090  2.199654  -7.140589   
38598 -14.191219 -34.104544  16.901802 -40.950516  2.887262 -18.701303   
38599 -17.816974 -41.019500   9.237521 -43.149826 -1.189121 -18.299228   
38600 -17.421202 -25.466024   6.805135 -23.326278 -8.085146 -11.039532   
38601 -14.485930 -42.760992   3.770227 -49.028301 -4.210399 -19.930272   

              11         12        13         14        15         16  \
38597 -11.371419  -6.316325 -1.074270 -30.207474 -0.527082  19.643021   
38598 -14.527838 -14.820559 -0.348226 -35.621625  1.847583  25.954240   
38599 -15.350239 -10.431189 -0.767667 -35.746382  3.343094  23.174434   
38600 -13.769865  -2.120747 -0.390735 -32.651667 -5.318570  33.078375   
38601 -14.495330  -9.914022 -0.962046 -28.493293  4.923430  31.080568   

              17         18         19         20         21        22  \
38597 -39.102989 -12.497799 -17.130351  19.062780 -13.586672  2.956176   
38598 -55.340111 -14.917046 -34.968337   9.749450 -40.573440  2.042546   
38599 -49.123892  -5.101466 -34.806061   5.864029 -41.285758  4.234043   
38600 -59.936537 -12.354740 -29.111430  25.449585 -28.205486  5.887074   
38601 -35.238489  10.090848 -18.162792  11.933037 -23.881296  6.350771   

              23         24         25        26         27        28  \
38597  -7.730684 -10.024323  -5.445839 -0.014795 -33.091600 -1.967639   
38598 -13.108698 -12.024891 -10.921892  0.344688 -33.182167  0.469917   
38599  -9.896289 -11.276410  -7.807656 -0.167643 -31.691120  2.242440   
38600 -13.445059 -15.129422 -13.168418  0.333417 -32.009480 -8.675913   
38601  -3.942249  -9.820903  -4.865081 -0.211735 -22.737588 -1.949838   

              29         30         31         32         33         34  \
38597  25.381388 -55.486240 -21.352537 -25.016082  27.214912 -21.580011   
38598  27.109812 -48.644167  -2.759899 -15.212965  25.102147 -24.363840   
38599  24.767940 -34.335007  12.479196 -18.360928  18.960557 -24.800241   
38600  29.889448 -59.778876 -19.199590 -26.223939  26.270631 -18.723416   
38601  21.602667 -22.191837  13.570110 -14.049456   6.929306 -13.769037   

             35         36         37         38        39         40  \
38597  7.102680  -7.808774 -13.044772 -13.562920  0.871483 -33.209238   
38598  0.463993 -15.190995  -9.871070 -15.901709  0.718849 -32.786350   
38599  4.238951  -9.846272  -8.720100  -6.618163  0.360284 -36.269642   
38600  4.168095  -8.613036 -12.885143  -9.379287  0.734289 -27.316493   
38601  7.737944   1.043642  -5.608355   3.457493  0.538511 -33.957005   

              41         42         43         44         45         46  \
38597  -8.578632  19.903485 -52.042726 -19.522709 -20.811197  30.936408   
38598  -2.778686  26.265287 -55.604312 -16.350376 -22.226949  22.088017   
38599   7.443324  30.833844 -50.356364  -8.263220 -19.803473  24.325671   
38600 -12.752364  28.003427 -47.378583 -11.096998 -14.236537  26.145712   
38601  -4.564327  31.127357 -59.769515 -14.329383 -24.872808  22.790730   

              47        48         49         50         51        52  \
38597 -13.884756  8.759265  -5.379706 -11.609683 -14.013027  1.333837   
38598 -19.779816  6.657963 -10.657367 -15.951945 -12.921225  1.003681   
38599 -22.789761  5.085009  -7.257866 -16.200006  -7.780213  0.777642   
38600 -13.002000 -1.631530   0.239814  -5.563313  -6.486615  0.857409   
38601 -24.326171  6.393690 -15.131431 -14.705015 -13.321660  1.087330   

              53    

In [46]:
def create_test_train(x_data, save_path, data_header):
    from sklearn.model_selection import train_test_split

    X_train, X_test = train_test_split( x_data, test_size=0.15, random_state=42, shuffle=True)

    print('Size of training matrix:', X_train.shape)
    print('Size of testing matrix:', X_test.shape)

    # test data
    np.savetxt('data/{}/{}.csv'.format(save_path,save_path+'_train'), X_train, delimiter=',', header=data_header)
    np.savetxt('data/{}/{}.csv'.format(save_path,save_path+'_test'), X_test, delimiter=',', header=data_header)

In [47]:
create_test_train(x_data, save_path, data_header)

Size of training matrix: (32811, 2809)
Size of testing matrix: (5791, 2809)
